# Advanced SQL - Part B

In today's class, we will learn how to extract timestamps, query multiple tables at once, and use subqueries to take full advantage of SQL. We will use IMDB and European Soccer databases for this course.

## What you will learn in this course 🧐🧐

- Extract some of the timestamps
- Know the difference between primary and secondary keys
- Know the difference between `INNER JOIN` and `OUTER JOIN` and how to write them.
- Make subqueries (or nested queries)
- Storing queries with `WITH AS`


## `EXTRACT` ✨

Before doing an `EXTRACT`, we will need to switch to Standard SQL. To do this:

* go to your Dataset `european_soccer`
* click on *show options* then uncheck *Use Legacy SQL*.

 We will leave it like this for the rest of the course.

When working with timestamps it is useful to know how to extract only the data you are interested in (often the date and not the time). For example, if we wanted to find the number of Matches per year, we could use the following query :

```sql
SELECT EXTRACT(year FROM date) as year, COUNT(*) as cnt
FROM european_soccer.Match
GROUP BY year
ORDER BY cnt DESC;
```

Generally speaking, the request is broken down as follows:

`EXTRACT([year, month, day] FROM timestamp_column) as alias`

If you don't give an alias to your `EXTRACT`, you will see a `f0_` appear as the column name. In general, it is clearer to give a more consistent alias such as *day*, *month*, *year* or something else.

## Primary Keys 🔑

A *primary key* is a column that serves as an identifier for each row. Each identifier is unique and is represented by an "_integer_" that increments each time a new row is created. Every table must have a primary key.

In the users table, the primary key corresponds to the ID of each user. Here is an example:

```sql
SELECT *
FROM european_soccer.Player
WHERE id = 1449;
```

In our european_soccer database we have several tables, each with its own primary key. Try the following queries:

* Find the country with ID 17642.
* Now find the league that has ID 4769
* Finally find the team that has the ID 43042

Do you see the difference?

## Foreign Keys 🗝️

A foreign key (or _foreign key_) is a column that contains the primary key of another table. We use both foreign keys and primary keys to connect tables together. Unlike primary keys, foreign keys are not unique and can also be NULL. A table can also have several foreign keys.

Let's take an example, starting from our very first table Food*truck (see Introduction to SQL part A). For example, we could have created a new table that lists all the banking transactions that have been made in each food truck. In this way, we could have calculated the income of each of them. Our \_transactions* table could have had several columns, one of which would have been the **ID** of the food truck so that we could link the transaction to the right food truck.

Now let's take our Match table from the European_soccer base. What do you see as foreign keys?

We have for example `country_id` and `league_id` as foreign keys. Be careful though, the `team_api_id` and `match_api_i` are foreign keys from the Match table but also from the Team table. This can be confusing because they are actually Id's too. We will explain this during the course.

## `INNER JOIN`

Joins in SQL are clauses that allow you to combine rows from several tables at the same time. The most commonly used join is the `INNER JOIN`, so much so that it has become the default join in most DBMSs.

An `INNER JOIN` will combine the rows of several tables according to a specific condition. Let's take an example:

```sql
SELECT name, COUNT(*) as cnt
FROM european_soccer.Country as Country
INNER JOIN european_soccer.Match as Match
ON Country.id = Match.country_id
GROUP BY name
ORDER BY cnt DESC;
```

`JOIN european_soccer.Match as Match ON` specifies the type of `JOIN` you want to make and the second table you want to join. So in this example we want to join the _Country_ table to the _Match_ table.

You will have to explicitly say which columns match the right keys. That's why you have this dot `.` which separates the table alias from the column name. Here's the generic notation: **alias_table.column_name**

`Country.id = Match.country_id` is the join condition. In other words, SQL will look for the _id_ of the _Country_ table that matches the _country_id \_of the \_Match_ table. Each time the numbers match, the corresponding rows will appear in your results.

The `INNER JOIN`s are so common that you can shorten them by simply putting JOIN:

```sql
SELECT name, COUNT(*) as cnt
FROM european_soccer.Country as Country
JOIN european_soccer.Match as Match
ON Country.id = Match.country_id
GROUP BY name
ORDER BY cnt DESC;
```

So now we know which countries have had the most matches. Let's try to see which country each league belongs to.

```sql
SELECT Country.name AS country_name, League.name AS league_name
FROM european_soccer.Country as Country
JOIN european_soccer.League as League
ON Country.id = League.country_id;
```

## `OUTER JOIN`

Although you will use the INNER JOIN most of the time, the `OUTER JOIN` can also be useful. These `OUTER JOIN`s also include IDs that do not match in the table you want. For example, in the IMDB, if you want a list of all tertiary actors, including those who have not been in a movie, and all movies, including those without tertiary actors, you can use an `OUTER JOIN`. In Google Big Query, the `FULL OUTER JOIN`  clause will be used :

```sql
SELECT actor_3_name, movie_title
FROM IMDB.third_actors AS Third_actor
FULL OUTER JOIN IMDB.movies AS Movies
ON Movies.actor_3_id = Third_actor.actor_3_id;
```

With this request, we get in return all the ids of each table that do not match in addition to those that do. In other words, in addition to the results of the INNER JOIN query, this query returns the tertiary actors who did not play in a movie, as well as the movies that do not have a tertiary actor.

If we add the `WHERE actor_3_name IS NULL` condition at the end, the query only returns movies that don't have a tertiary actor, or an actor whose id is not present in the `IMDB.third_actors` table. Similarly, if we add the `WHERE movie_title IS NULL` condition at the end, the query only returns tertiary actors who have no third actors.

```sql
SELECT actor_3_name, movie_title
FROM IMDB.third_actors AS Third_actor
FULL OUTER JOIN IMDB.movies AS Movies
ON Movies.actor_3_id = Third_actors.actor_3_id
WHERE movie_title IS NULL;
```

If we had done an `INNER JOIN`, we wouldn't have had any results. You can give it a try:

```sql
SELECT actor_3_name, movie_title
FROM IMDB.third_actors AS Third_actors
INNER JOIN IMDB.movies AS Movies
ON Movies.actor_3_id = Third_actors.actor_3_id
WHERE movie_title IS NULL;
```

This time we get “_Query returned zero records._”

However, we do not have to have both tables complete. For example, you can want all the tertiary actors, including those who haven't been in a film, but you don't want all the films in which there were no tertiary actors. That's where the line between `LEFT JOIN` and `RIGHT JOIN` comes in. These clauses will define in which SQL table will output all the info (including the IDs that don't match). Most of the time, only `LEFT JOIN` will be used, because `RIGHT JOIN` is not always included in all RDBMSs. Here is an example:

```sql
SELECT actor_3_name, movie_title
FROM IMDB.third_actors AS Third_actors
LEFT JOIN IMDB.movies AS Movies
ON Movies.actor_3_id = Third_actors.actor_3_id;
```

This query returns all tertiary actors in the `IMDB.Third_actors` table, including those who have not been in a movie.

Conversely, we could make the same query by performing a `RIGHT JOIN` :

```sql
SELECT actor_3_name, movie_title
FROM IMDB.third_actors AS Third_actors
RIGHT JOIN IMDB.movies AS Movies ON Movies.actor_3_id = Third_actors.actor_3_id;
```

We then get all the movies from the `IMDB.movies` table, including those without a tertiary actor. This is like doing a `LEFT JOIN' by flipping the two tables:

```sql
SELECT actor_3_name, movie_title
FROM IMDB.movies AS Movies
LEFT JOIN IMDB.third_actors AS Third_actors
ON Movies.actor_3_id = Third_actors.actor_3_id;
```

NB: If the columns you want to make your `JOIN` with have the same name, you can use the USING() function:

```sql
SELECT actor_3_name, movie_title
FROM IMDB.third_actors AS Third_actors
INNER JOIN IMDB.movies AS Movies
USING(actor_3_id);
```

## Multiple `JOIN`s

If you need to make a multiple join, you can simply add `JOIN` lines one after the other in the query. In the example below, we do a join on the main, secondary and tertiary movies and actors:

```sql
SELECT actor_1_name, actor_2_name, actor_3_name, movie_title
FROM IMDB.third_actors AS Third_actors
JOIN IMDB.movies AS Movies
USING(actor_3_id)
JOIN IMDB.main_actors AS Main_actors
USING(actor_1_id)
JOIN IMDB.second_actors AS Second_actors
USING(actor_2_id);
```

## `WITH AS`

If you need to make a subquery, it is good to store it somewhere so you don't have to rewrite it. The `WITH AS` clause allows you to do this. Let's look at an example:

```sql
WITH imdb_score_in_USA AS (
SELECT imdb_score
FROM IMDB.movies
WHERE Country = "USA")
SELECT avg(imdb_score)
FROM imdb_score_in_USA;
```

With the query "_imdb_score_in_USA_" all IMDB scores of American movies are selected. So you can reuse this query as you wish!

Be careful though, with the `WITH AS` clause, it's as if you were creating a new table. You can use this query only after a FROM or in a JOIN.

## `CASE`

An interesting clause to know is the `CASE` clause. It allows you to return a value according to a given condition. It has the following logical structure:

_If CONDITION Then VALUE_

In this case, however, we will have the following values:

```sql
CASE
    WHEN condition1 THEN value1
    WHEN condition2 THEN value2
    ELSE value3
END
```

For example:

```sql
SELECT movie_title, duration,
CASE
    WHEN duration < 80 THEN 'Short Duration'
    WHEN duration > 81 AND duration < 120 THEN 'Average Duration'
    ELSE 'Long Duration'
END AS renamed_duration
FROM IMDB.movies;
```

## Resources 📚📚

- Relational Queries - [http://bit.ly/1Sc6Ziu](http://bit.ly/1Sc6Ziu)
- Inner Joins - [http://bit.ly/2ivq7hR](http://bit.ly/2ivq7hR)
- Left Joins - [http://bit.ly/2zUghNC](http://bit.ly/2zUghNC)
- Multiple Joints - [http://bit.ly/2iAG3zj](http://bit.ly/2iAG3zj)